In [ ]:
import sys, os, glob, copy, re
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator, interp2d
import matplotlib as mpl
from matplotlib.colors import LogNorm
from IPython.display import display, Markdown
from collections import OrderedDict
import pylhe
import pyslha
import xml.etree.ElementTree as ET
from particle import Particle, PythiaID
#Particle.from_pdgid(211)
pythiaid = PythiaID(-4122)
p = Particle.from_pdgid(pythiaid.to_pdgid())
p.name


### Get hepMC (input) folder and extract model info

In [ ]:
# Define the directory and pattern for input files
inputDir = '/home/yoxara/Eff_MA5_LO'
#inputDir = '/media/yoxara/T9/Events_MG5_MA5_ATLAS-EXOT-2018-06_eff/Eff_MA5'
listPattern = os.path.join(inputDir, '_run_*/Input/*.list')
listFiles = sorted(glob.glob(listPattern))

# Define the pattern for output files
effPattern = os.path.join(inputDir, '*_run_*/Output/SAF/spin1_run_*/CLs_output.dat')
effFiles = sorted(glob.glob(effPattern))

# Function to extract the run number from a file path
def extract_run_number(file_path):
    match = re.search(r'_run_(\d+)', file_path)
    return int(match.group(1)) if match else None

# Sort files by their run numbers
sorted_listFiles = sorted(listFiles, key=extract_run_number)
sorted_effFiles = sorted(effFiles, key=extract_run_number)

for input_file, output_file in zip(sorted_listFiles, sorted_effFiles):
    input_run = extract_run_number(input_file)
    output_run = extract_run_number(output_file)
    print(f"Input Run: {input_run}, Output Run: {output_run}")
    print(f"Input file: {input_file}")
    print(f"Output file: {output_file}\n")




In [ ]:
print(f"Found {len(sorted_listFiles)} .list files") 


In [ ]:
allDataDicts = []

for inputFile, effFile in zip(sorted_listFiles, sorted_effFiles ):
    with open(inputFile, 'r') as f:
        hepMCfile = f.read().strip()
        hepDir = os.path.dirname(hepMCfile)
        print('HepMC input dir=',hepDir)
    
    try:
        banner = sorted(glob.glob(hepDir+'/*banner.txt'), key=os.path.getmtime, reverse=True)
        if len(banner) == 0:
            print('Banner not found for %s' % inputFile)
            continue
        elif len(banner) > 1:        
            print('\n%i banner files found for %s. Using %s' 
                  % (len(banner), inputFile, os.path.basename(banner[0])))
        banner = banner[0]
        
        xtree = ET.parse(banner)
        xroot = xtree.getroot()
        genInfo = xroot.find('header').find('MGGenerationInfo').text.strip().split('\n')
        genInfo = [x.replace('#', '').strip().split(':') for x in genInfo]
        nevts = [eval(x[1]) for x in genInfo if 'Number of Events' in x[0]][0]
        xsecPBall = [eval(x[1]) for x in genInfo if 'Integrated weight (pb)' in x[0]]
        xsecPBmatched = [eval(x[1]) for x in genInfo if 'Matched Integrated weight (pb)' in x[0]]
        if xsecPBmatched:
            nevts = nevts * (xsecPBmatched[0] / xsecPBall[0])
            xsecPB = xsecPBmatched[0]
        else:
            xsecPB = xsecPBall[0]

        slha = xroot.find('header').find('slha').text
        pars = pyslha.readSLHA(slha)
        mMed = pars.blocks['MASS'][55]
        mDM = pars.blocks['MASS'][52]
        gVq = pars.blocks['DMINPUTS'][4] # Mediator-quark vector coupling
        gAq = pars.blocks['DMINPUTS'][10] # Mediator-quark axial coupling
        gVx = pars.blocks['DMINPUTS'][2] # Mediator-DM vector coupling
        gAx = pars.blocks['DMINPUTS'][3] # Mediator-DM axial coupling
        print('Cross-section (pb) = %1.3e' % xsecPB)
        print('Number of Events = %i' % nevts)
        print('mMed = %1.2f GeV, mDM = %1.2f GeV, gVq = %1.2f, gAq = %1.2f, gVx = %1.2f, gAx = %1.2f' 
              % (mMed, mDM, gVq, gAq, gVx, gAx))
    except Exception as e:
        print(f"Error processing banner for {inputFile}: {e}")
        continue
    
    dataDict = {}
    if banner:
        dataDict['filename'] = hepMCfile
        dataDict['Total xsec (pb)'] = xsecPB
        dataDict['Total MC Events'] = nevts
        if gVx != 0:
            dataDict['Coupling'] = 'Vector'
        else:
            dataDict['Coupling'] = 'Axial'

        dataDict['Mode'] = 'DM+QCDjets'

        dataDict['$m_{med}$'] = mMed
        dataDict['$m_{DM}$'] = mDM
        if dataDict['Coupling'] == 'Vector':
            dataDict['$g_{DM}$'] = gVx
            dataDict['$g_{q}$'] = gVq
        else:
            dataDict['$g_{DM}$'] = gAx
            dataDict['$g_{q}$'] = gAq
        
    with open(effFile, 'r') as f:
        for il, l in enumerate(f.readlines()):
            if il == 0:
                l = l.replace('#', '')
                header = [x.strip() for x in l.split('  ') if x.strip()]
                dataDict.update({col : [] for col in header})
            else:
                pt = [None] * len(dataDict)
                if not l.split():
                    continue
                for ix, x in enumerate(l.split()):
                    if not x.strip():
                        continue
                    try:
                        x = eval(x)
                    except:
                        pass
                    if ix > len(dataDict) - 1:
                        continue
                    pt[ix] = x
                for icol, col in enumerate(header):
                    dataDict[col].append(pt[icol])
    
    if '||' in dataDict:
        dataDict.pop('||')

    allDataDicts.append(dataDict)


In [ ]:
# Combine all data into a single DataFrame
all_df = pd.concat([pd.DataFrame.from_dict(d) for d in allDataDicts], ignore_index=True)
all_df.head()


In [ ]:
output_csv = 'efficiencyMap_Combined.csv'
all_df.to_csv(output_csv, index=False)


In [ ]:
file_path = 'efficiencyMap_Combined.csv'
data = pd.read_csv(file_path)
bins = data['signal region'].unique()
for bin_name in bins:
    bin_data = data[data['signal region'] == bin_name]
    file_name = f'efficiencyMap__SR_{bin_name}.csv'
    bin_data.to_csv(file_name, index=False)
    print(f"Data for {bin_name} saved to {file_name}")


In [ ]:
columns_to_keep = ['$m_{med}$', '$m_{DM}$', 'efficiency']
for bin_name in bins:
    bin_data = data[data['signal region'] == bin_name][columns_to_keep]
    #file_name = f'../data/orig/data_eff/axial-vector_new/efficiencyMap__SR_{bin_name}.csv'   
    file_name = f'/home/yoxara/smodels-database/13TeV/ATLAS/ATLAS-EXOT-2018-06_eff/orig/axial_vector/efficiencyMap_SR_{bin_name}.csv'
    with open(file_name, 'w') as f:
        f.write('# ' + ','.join(columns_to_keep) + '\n')
        bin_data.to_csv(f, index=False, header=False)
    print(f"Data for {bin_name} saved to {file_name}")
